In [ ]:
!pip install tsfresh
!pip install shap
!pip install catboost

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from collections import Counter
from tqdm import tqdm
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
import scipy.fftpack

import catboost
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp

import lightgbm as lgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score


In [ ]:
!wget https://onti-2019.s3-eu-central-1.amazonaws.com/public/data_v1.zip
!unzip data_v1.zip

In [ ]:
transactions_train=pd.read_csv('data/transactions_train.csv')
transactions_test=pd.read_csv('data/transactions_test.csv')

In [ ]:
transactions = transactions_train.append(transactions_test)

In [ ]:
transactions['amount_rur'] = np.log(transactions['amount_rur'])

In [ ]:
params = ['mean', 'sum', 'count', 'median', 'std', 'min', 'max', lambda x: x.max() - x.min()]

In [ ]:
main_fea= transactions.groupby('client_id')['amount_rur'].agg(params)
main_fea.columns =  ['main_fea_' + str(i) for i in main_fea.columns]

In [ ]:
temp = transactions.groupby(['client_id', 'small_group'])['amount_rur'].agg(params)

temp2 = pd.DataFrame()
for col in temp.columns:
  temp2 = pd.concat([temp2, temp.reset_index().pivot(index='client_id', columns='small_group', values=col)], axis=1)

temp2 = temp2.fillna(0)

temp2.columns = ['sm_fea_' + str(i) for i in temp2.columns]

main_fea = pd.merge(main_fea, temp2, on='client_id', how='outer')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/My Drive/'

/content/drive/My Drive


In [ ]:
temp = pd.read_csv('data_ntiai.csv')

In [ ]:
temp.shape

(50000, 749)

In [ ]:
temp = temp[['amount_rur__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"intercept"',
       'amount_rur__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"',
       'amount_rur__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"intercept"',
       'amount_rur__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
       'amount_rur__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
       'amount_rur__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',
       'amount_rur__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',
       'amount_rur__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
       'amount_rur__index_mass_quantile__q_0.2',
       'amount_rur__longest_strike_below_mean', 'amount_rur__median',
       'amount_rur__minimum', 'amount_rur__number_crossing_m__m_1',
       'amount_rur__percentage_of_reoccurring_datapoints_to_all_datapoints',
       'amount_rur__quantile__q_0.1', 'amount_rur__quantile__q_0.6',
       'amount_rur__quantile__q_0.7', 'amount_rur__quantile__q_0.8',
       'amount_rur__range_count__max_1__min_-1',
       'amount_rur__ratio_value_number_to_time_series_length',
       'amount_rur__sum_of_reoccurring_data_points',
       'amount_rur__sum_of_reoccurring_values']]

In [ ]:
temp.index = main_fea.index

In [ ]:
main_fea = pd.merge(main_fea, temp, on='client_id', how='outer')

In [ ]:
main_fea = main_fea.fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler
ind = main_fea.index
min_max_scaler = StandardScaler()
main_fea_s = min_max_scaler.fit_transform(main_fea.values)
main_fea_s = pd.DataFrame(main_fea_s)
main_fea_s.index = ind

In [ ]:
main_fea_s.columns = ['f' + str(i) for i in range(len(main_fea_s.columns))]

In [ ]:
train_target = pd.read_csv('/content/data/train_target.csv')
TRAIN = main_fea_s.iloc[train_target['client_id']]
TEST = main_fea_s.iloc[list(set(main_fea_s.index.values) - set(train_target['client_id']))]

In [ ]:
y=train_target['bins']
X=TRAIN

In [ ]:
from hyperopt.pyll.base import scope
@scope.define
def to_int(x):
    return int(x)

In [ ]:
ctb_params = {
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.06),
    'max_depth': scope.to_int(hp.quniform('max_depth', 2, 10, 1)),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 2, 6),
    'num_trees': scope.to_int(hp.quniform('num_trees', 2000, 15000, 1)),
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 2),
    'random_strength': hp.uniform('random_strength', 0, 2),

    'eval_metric': 'Accuracy',
    'loss_function': 'MultiClass', 
    'task_type': 'GPU', 
    'thread_count': -1, 
    'verbose': False,
}


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=2017)
xx = X.values

In [ ]:
class HPOpt(object):
    def __init__(self, xx, y):
        self.xx = xx
        self.y = y

    def process(self, params, algo, max_evals, trials):
        try:
            result = fmin(fn=self.train_reg, space=params, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            print(e)
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def train_reg(self, params):
      scores = []
      modelsm = []

      print(params)

      for train_index, test_index in kf.split(xx):
        X_train, X_test = xx[train_index], xx[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        res = accuracy_score(y_test, y_pred)

        print(res)

        scores.append(res)
        modelsm.append(model)

      score_m = sum(scores)/len(scores)
      print(score_m)

      return {'loss': 1-score_m, 'status': STATUS_OK}

In [ ]:
obj = HPOpt(xx, y)
lgb_opt = obj.process(params=ctb_params, algo=tpe.suggest, max_evals=50000, trials=Trials())

In [ ]:
best_params = {'bagging_temperature': 1.822557323737978, 'eval_metric': 'Accuracy', 'l2_leaf_reg': 4.59752078934112, 'learning_rate': 0.021125893198394825, 'loss_function': 'MultiClass', 'max_depth': 6, 'num_trees': 9553, 'random_strength': 0.2522752957559110, 'task_type': 'GPU', 'thread_count': -1, 'verbose': False, }

In [ ]:
scores = []
modelsm = []

for train_index, test_index in kf.split(xx):
  X_train, X_test = xx[train_index], xx[test_index]
  y_train, y_test = y[train_index], y[test_index]

  model = CatBoostClassifier(**best_params)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  res = accuracy_score(y_test, y_pred)

  print(res)

  scores.append(res)
  modelsm.append(model)

score_m = sum(scores)/len(scores)
print(score_m)

In [ ]:
best_params = {'bagging_temperature': 1.822557323737977, 'eval_metric': 'Accuracy', 'l2_leaf_reg': 4.59752078934111, 'learning_rate': 0.021125893198394825, 'loss_function': 'MultiClass', 'max_depth': 6, 'num_trees': 9553, 'random_strength': 0.2522752957559109, 'task_type': 'GPU', 'thread_count': -1, 'verbose': False, }

In [ ]:
model = CatBoostClassifier(**best_params)
model.fit(X, y)

In [ ]:
y_pred = model.predict(TEST)

In [ ]:
y_pred.shape

(20000, 1)

In [ ]:
sub = pd.DataFrame(y_pred, columns=['bins'])
sub.index = TEST.index

submission = sub

In [ ]:
y_pred[0]

2.0

In [ ]:
submission = pd.DataFrame({'bins': y_pred},index=TEST.index)
submission

In [ ]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv('data/sub.csv', index=True)

In [ ]:
submission = submission.reset_index().rename({'index':'client_id'},axis=1)
submission

,client_id,bins
0,0,2.0
1,1,1.0
2,2,0.0
3,3,0.0
4,5,1.0
...,...,...
19995,32757,3.0
19996,32758,2.0
19997,32762,2.0
19998,32764,1.0
